In [2]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
%matplotlib inline




class BMC_data():
    """
    An object created from BMC particle data, 
    represented by a Pandas dataframe and manipulations thereof

    """
    
    def __init__(self,BMC_file):    

        self.table = pd.read_table(BMC_file,delim_whitespace=True)
        self.table_size = len(self.table['x'])
        self.truth = self.table.isna()
        self.particleNumber = np.count_nonzero(self.truth['y'])
    
    
        ####################################################################
        ####################################################################
        """
        1) Takes the data in its table form
        2) looks for locations in the data where a new particle begins
        3) notes the new particle's row index in the table (particleLocationArray)
        4) notes how many data points/rows are associated with that particle (particleEventArray) 
        """
        particle_num_index = []
        events_for_particle = []
    
        for index in np.arange(self.table_size):
        
            if self.truth['y'][index]:
                events_for_particle.append(self.table['x'][index])
                particle_num_index.append(index)
        
        
        self.particleLocationArray = particle_num_index
        self.particleEventArray = events_for_particle
        
        ####################################################################
        ####################################################################
 
    def filter_array(self,array,zscore=1.5):
        
        data = np.array(array)
        
        sDev = np.std(data)
        mean = np.mean(data)
        
        centered_data = (data-mean)/sDev
        
        filtered_data = np.array([entry for entry in centered_data if np.abs(entry) <= zscore])
        
        converted_back = filtered_data*sDev+mean
        
        return converted_back
    
    
    def pull_particle(self,whichParticle,skipFirstLine = False):
        """    
        Return dataframe of only whichParticle's data
    
        for N particles in a set of data, 
        0 <= whichParticle <= N
        (whichParticle is an integer, e.g. whichParticle = 2)
        """      
        first_index = int(self.particleLocationArray[whichParticle] + 1 + skipFirstLine)
        final_index = int(first_index + self.particleEventArray[whichParticle] - skipFirstLine)
    
        return self.table[first_index:final_index]
    

    def convert_units(self,conversion_factor):
        """
        Given a conversion factor, scales the spatial data by that much
        ##### Only use this if you didn't scale your data in the BMC software #####
        ##### Only use this once or you'll over/underscale your data ######
        
        """
        rows_with_data = self.truth['y'] == False
        
        self.table['x'][rows_with_data] = self.table['x']*conversion_factor
        self.table['y'][rows_with_data] = self.table['y']*conversion_factor
        self.table['dx'][rows_with_data] = self.table['dx']*conversion_factor
        self.table['dy'][rows_with_data] = self.table['dy']*conversion_factor
        
        self.table['dr^2'][rows_with_data] = self.table['dr^2']*conversion_factor**2
        self.table['TotalDisplacementSquared'][rows_with_data] = self.table['TotalDisplacementSquared']*conversion_factor**2
        
        return self.table

    def plot_trajectory(self, whichParticle,xLabel='x position',yLabel='y position'):

        data = self.pull_particle(whichParticle)
        
        x_data = data['x']
        y_data = data['y']
    
        plt.plot(x_data,y_data,lw=2)
        plt.xlabel(xLabel)
        plt.ylabel(yLabel)
        
        
    def plot_trajectories(self):   
        for particle in np.arange(self.particleNumber):
            self.plot_trajectory(whichParticle=particle)
    
    def get_avg_velocity(self):
        """
        Return the average velocity of every particle in a numpy array
        (net displacement/change in time)
        """
        velocities = []

        for particle in np.arange(self.particleNumber):
        
            particleData = self.pull_particle(whichParticle=particle)      
            
            totalTime = np.sum(particleData['dt'])
            distances = list(particleData['TotalDisplacementSquared'])
            
            final_distance = distances[-1:][0]**0.5
        
            avgVelocity = final_distance/totalTime
            
            velocities.append(avgVelocity)
    
        return np.array(velocities)    
    
    def plot_displacement_single(self,whichParticle,xLabel='time (s)',yLabel='Displacement Squared (units squared)', size = 10):
        
        '''
        Takes in a single particle by its number (integer)
        Optional: rename the x and y axes (meters or pixels), change dot size
        
        Return a plot of that particle's net squared displacement versus time
        '''
        
        data = self.pull_particle(whichParticle)
        
        displacements = data['TotalDisplacementSquared']
        time = data['time']
        
        plt.plot(time,displacements)
        
        plt.xlabel(xLabel)
        plt.ylabel(yLabel)
        
    
    def plot_displacement_all(self,xLabel='time (s)',yLabel='Displacement Squared (units squared)', size = 10):
        
        for particle in np.arange(self.particleNumber):
            self.plot_displacement_single(particle,xLabel=xLabel,yLabel=yLabel, size = size)
        
        
    def compute_diffusionSingle(self,whichParticle):
        
        data = self.pull_particle(whichParticle)
        
        numerator = np.sum(data['TotalDisplacementSquared']*data['time'])
        
        denominator = 4*np.sum(data['time']*data['time'])
        
        return numerator/denominator
        
    def compute_diffusionAll(self):
        
        diffusion_coefficients = []
        weights = self.particleEventArray/sum(self.particleEventArray)
        
        for particle in np.arange(self.particleNumber):
            
            diffusion_coefficient = self.compute_diffusionSingle(whichParticle=particle)
            diffusion_coefficients.append(diffusion_coefficient)
        
        weighted_total_diffusion = sum(diffusion_coefficients*weights)
        
        return weighted_total_diffusion
    
    def avgInstantaneous_velocitySingle(self,whichParticle):
        
        data = self.pull_particle(whichParticle,skipFirstLine=True)
        
        dxdt = data['dx']/data['dt']
        dydt = data['dy']/data['dt']
        
        magnitudes = np.array((dxdt**2 + dydt**2)**0.5)
        
        return np.mean(magnitudes)
    
    def avgInstantaneous_velocityAll(self,filterData=True,zscore=2):
        
        avg_vels = np.array([])

        for particle in np.arange(self.particleNumber):
            avg_vels = np.append(avg_vels,self.avgInstantaneous_velocitySingle(particle))
            
        if filterData:
            avg_vels = self.filter_array(avg_vels,zscore=zscore)
        
        return avg_vels